In [1]:
!pip install pandahouse -q

  Preparing metadata (setup.py) ... done


In [2]:
import pandahouse as ph

In [3]:
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')


In [4]:
def select(sql):
    return ph.read_clickhouse(sql, connection=connection)

Теперь описание данных:

installs — содержит данные об установках приложения по дням.

DeviceID — идентификатор устройства, на которое было установлено приложение;
InstallationDate — дата установки приложения;
InstallCost — цена установки приложения в рублях;
Platform — платформа, на которой было установлено приложение (iOS/ Android);
Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).
events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

DeviceID — идентификатор устройства, на котором используется приложение;
AppPlatform — платформа, на которой используется приложение (iOS/ Android);
EventDate — дата, за которую собрана статистика;
events — количество просмотров всех товаров за этот день у этого DeviceID.
checks — содержит данные о покупках пользователей в приложении по дням

UserID — идентификатор пользователя;
Rub — суммарный чек пользователя на дату;
BuyDate — дата, за которую собрана статистика.
Особенность приложения заключается в том, что для просмотра товаров не нужна авторизация. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices:

DeviceID — идентификатор устройства;
UserID — идентификатор пользователя.

## Задача 4

Продолжим работать с таблицей checks!

Посчитайте, сколько покупок приходится на каждого клиента, указав имя столбца как NumChecks. Результирующую таблицу с UserID, NumChecks отсортируйте по убыванию (DESC) нового столбца. Ограничение вывода – 10.

In [11]:
sql = ''' select 

t.UserID,
count(t.BuyDate) as NumChecks

from checks as t

group by t.UserID

order by NumChecks DESC

limit 10
'''

In [12]:
select(sql)

,UserID,NumChecks
0,12506794130929660079,276
1,640807012715027711,276
2,9851217944924628313,274
3,3220599270443999977,272
4,109604902333456192,270
5,17587339617847831868,270
6,8752522755161623524,269
7,6295159848944215572,267
8,5893749085932398221,266
9,10032596276912502803,266


## Задание 5

Давайте немного усложним задачу, и посчитаем ещё и сумму сделанных покупок в рублях (Rub).

Выведите в результирующую таблицу UserID клиентов, количество сделанных покупок (NumChecks) и сумму потраченных денег (Revenue). Результат отсортируйте по убыванию Revenue, оставив 10 записей.

In [15]:
sql = ''' select 

t.UserID,
count(t.BuyDate) as NumChecks,
sum(t.Rub) as Revenue

from checks as t

group by t.UserID

order by Revenue DESC

limit 10
'''

In [16]:
select(sql)

,UserID,NumChecks,Revenue
0,15605251414578189718,3,181146
1,16967914029594691558,10,139992
2,9613023063681218234,51,70128
3,11863769084622496656,32,63193
4,958354669642668056,49,60196
5,6911136753844832704,79,57721
6,10398072986532593332,21,57301
7,8208688783013614056,114,51605
8,5471859434812058036,82,46470
9,12791764976753443152,70,45662


## Задание 6

Теперь давайте попробуем с помощью группировки и агрегатных функций посмотреть динамику трат клиентов. Для этого нужно посчитать по дням минимальный, средний, и максимальный чек. Иными словами, необходимо сгруппироваться по датам, и использовать агрегатные функции MIN, AVG, MAX.

Выведите минимальный (MinCheck), максимальный (MaxCheck) и средний (AvgCheck) чек по дням. В результирующей таблице используйте сортировку по убыванию (DESC) столбца BuyDate. Как и на предыдущем шаге, ограничение на вывод равно 10.

In [17]:
sql = '''select
t.BuyDate,
min(t.Rub) as MinCheck,
max(t.Rub) as MaxCheck,
avg(t.Rub) as AvgCheck

from checks as t

group by t.BuyDate

order by t.BuyDate desc

limit 10
'''

In [18]:
select(sql)

,BuyDate,MinCheck,MaxCheck,AvgCheck
0,2019-10-04,0,3239,11.054391
1,2019-10-03,0,4898,10.357468
2,2019-10-02,0,3652,9.743485
3,2019-10-01,0,2928,9.813259
4,2019-09-30,0,4189,9.755777
5,2019-09-29,0,3213,11.740202
6,2019-09-28,0,4838,12.686164
7,2019-09-27,0,3232,11.156494
8,2019-09-26,0,7440,10.465333
9,2019-09-25,0,13906,10.394969


## Задание 7

Представим, что для проведения новой акции нам потребовалось выгрузить сегмент пользователей, которые приносят наибольшую выручку.

Найдите покупателей, сумма покупок которых превышает 10 000 рублей (Rub). Для этого сгруппируйте пользователей по UserID, затем примените нужную агрегатную функцию и назовите новую колонку Revenue. Отфильтруйте нужные значения, результат отсортируйте по убыванию UserID и выведите 10 строк.

In [19]:
sql = '''select
t.UserID,
sum(t.Rub) as Revenue

from checks as t

group by t.UserID

having Revenue > 10000

order by t.UserID desc

limit 10
'''

In [20]:
select(sql)

,UserID,Revenue
0,18266959643473967483,20395
1,18261532742663857859,13787
2,18105450744137216332,19858
3,17961878534766141140,17649
4,17907093951683732524,16556
5,17830166523985934832,11611
6,17789705480254581289,10834
7,17755116231546697738,26491
8,17717886684516945988,15055
9,17649746190216015646,22888


## Задание 8

В этой и последующих задачах мы будем работать с таблицей retail в ClickHouse, которая содержит данные о покупках. 

Задания выполняются в ClickHouse, в качестве ответа, как правило, нужно будет вписывать значения какого-либо поля из результирующей таблицы.

Как подключиться к Clickhouse

Таблица включает следующие поля: 

InvoiceNo — идентификатор покупки
StockCode — идентификатор товара на складе
Description — текстовое описание товара
Quantity — количество товара
InvoiceDate — дата покупки
UnitPrice — цена товара
CustomerID — идентификатор покупателя
Country — страна
Представим, что к вам пришел менеджер с проблемой: за последний месяц он наблюдает снижение выручки в некоторых регионах и хочет определить возможную причину этого снижения. Для начала можно сгруппировать данные о выручке по странам и найти среди них топ-5 стран по величине выручки, так как интереснее всего нам будет смотреть именно на данные этих стран.

Посчитайте выручку (Revenue) по странам и отсортируйте таблицу в порядке убывания выручки. В качестве ответа скопируйте название страны с самой высокой выручкой.  Обратите внимание, что в изначальной таблице указана цена одной единицы товара (UnitPrice) и количество купленного товара (Quantity), а не выручка, и для получения Revenue нужно будет использовать агрегирующую функцию Sum и оператор умножения *.

In [47]:
sql = '''SELECT
    t.Country,
    sum(t.Quantity * t.UnitPrice) as Revenue
    
FROM retail as t 

group by t.Country

order by Revenue desc

limit 5'''

In [48]:
select(sql)

KeyError: ignored

## Задание 10

Теперь посмотрим на динамику общей суммы выручки по месяцам.

Посчитайте выручку (Revenue) по месяцам (обратите внимание, что в таблице указана цена одной единицы товара и количество купленного товара), округлив InvoiceDate к началу месяца с помощью toStartOfMonth().

В качестве ответа укажите сумму выручки (Revenue) из первой строки результирующей таблицы, используя сортировку по убыванию (DESC) по столбцу Revenue. Не забывайте отфильтровать строки по условию Description != 'Manual'



In [49]:
sql ='''SELECT
    toStartOfMonth(t.InvoiceDate) as month,
    sum(t.Quantity * t.UnitPrice) as Revenue
    
FROM
    retail as t
    
WHERE
    t.Description != 'Manual'
    
GROUP BY 
    month
    
order by 
    Revenue desc'''